In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')


In [1]:
df=pd.read_csv('green-tripdata-2025-03.csv')
df.head()

NameError: name 'pd' is not defined

In [ ]:
print("Initial shape:",df.shape)
print("\nData types:\n",df.dtypes)
print("\nMissing values:\n",df.isnull().sum())

Initial shape: (51539, 21)

Data types:
 VendorID                   int64
lpep_pickup_datetime      object
lpep_dropoff_datetime     object
store_and_fwd_flag        object
RatecodeID                object
PULocationID               int64
DOLocationID               int64
passenger_count           object
trip_distance            float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
ehail_fee                 object
improvement_surcharge    float64
total_amount             float64
payment_type              object
trip_type                 object
congestion_surcharge      object
cbd_congestion_fee       float64
dtype: object

Missing values:
 VendorID                 0
lpep_pickup_datetime     0
lpep_dropoff_datetime    0
store_and_fwd_flag       0
RatecodeID               0
PULocationID             0
DOLocationID             0
passenger_count          0
trip_distance     

In [ ]:
df=df.dropna(subset=['lpep_pickup_datetime','lpep_dropoff_datetime','PULocationID','DOLocationID'])

In [ ]:
df['ehail_fee']=df['ehail_fee'].fillna(0)
df['improvement_surcharge']=df['improvement_surcharge'].fillna(df['improvement_surcharge'].median())
df['congestion_surcharge']=df['congestion_surcharge'].fillna(0)
df['cbd_congestion_fee']=df['cbd_congestion_fee'].fillna(0)


In [ ]:
df['lpep_pickup_datetime']=pd.to_datetime(df['lpep_pickup_datetime'])
df['lpep_dropoff_datetime']=pd.to_datetime(df['lpep_dropoff_datetime'])

In [ ]:
df['trip_duration']=(df['lpep_dropoff_datetime']-df['lpep_pickup_datetime']).dt.total_seconds()

In [ ]:
df=df[df['trip_duration']>0]

In [ ]:
df=df[df['fare_amount']>=0]

In [ ]:
df=df[df['trip_distance']>=0]

In [ ]:
def remove_outliers(df,column):
    Q1=df[column].quantile(0.25)
    Q3=df[column].quantile(0.75)
    IQR=Q3-Q1
    lower_bound=Q1-1.5*IQR
    upper_bound=Q3+1.5*IQR
    return df[(df[column]>=lower_bound)&(df[column]<=upper_bound)]

In [ ]:
df=remove_outliers(df,'trip_duration')
df=remove_outliers(df,'fare_amount')
df=remove_outliers(df,'trip_distance')
df=remove_outliers(df,'total_amount')

In [ ]:
df['speed_mph']=(df['trip_distance']/(df['trip_duration']/60))
df=df[df['speed_mph']<=100]

In [ ]:
same_loc_zero_dist = (df['PULocationID'] == df['DOLocationID']) & (df['trip_distance'] == 0)
df = df[~(same_loc_zero_dist & (df['total_amount'] > 10))]

In [37]:
df['pickup_hour'] = df['lpep_pickup_datetime'].dt.hour
df['pickup_day'] = df['lpep_pickup_datetime'].dt.day
df['pickup_dayofweek'] = df['lpep_pickup_datetime'].dt.dayofweek

In [39]:
df['is_weekend'] = df['pickup_dayofweek'].isin([5, 6]).astype(int)
df['is_rush_hour'] = ((df['pickup_hour'] >= 7) & (df['pickup_hour'] <= 10) | 
                     ((df['pickup_hour'] >= 16) & (df['pickup_hour'] <= 19))).astype(int)

In [41]:
columns_to_drop = ['ehail_fee', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'speed_mph']
df = df.drop(columns=columns_to_drop, errors='ignore')

In [43]:
df = df.reset_index(drop=True)

In [45]:
df.to_csv('cleaned_green_tripdata_2025_03.csv', index=False)

In [47]:
print("\nFinal shape after preprocessing:", df.shape)
print("\nSample of cleaned data:")
print(df.head())


Final shape after preprocessing: (41740, 24)

Sample of cleaned data:
   VendorID store_and_fwd_flag RatecodeID  PULocationID  DOLocationID  \
0         2                  N          1            75           239   
1         2                  N          1            41            42   
2         2                  N          1            66            33   
3         2                  N          1           134           122   
4         2                  N          1           260           260   

  passenger_count  trip_distance  fare_amount  extra  mta_tax  ...  \
0               1           2.20         18.4    1.0      0.5  ...   
1               1           1.06          8.6    1.0      0.5  ...   
2               1           0.82          5.8    1.0      0.5  ...   
3               1           4.94         21.9    1.0      0.5  ...   
4               1           0.81          8.6    1.0      0.5  ...   

   payment_type  trip_type  congestion_surcharge  cbd_congestion_fee 

In [49]:
df.tail()

,VendorID,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,...,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee,trip_duration,pickup_hour,pickup_day,pickup_dayofweek,is_weekend,is_rush_hour
41735,2,\N,\N,166,239,\N,1.92,14.54,0.0,0.5,...,\N,\N,\N,0.00,420.0,22,31,0,0,0
41736,2,\N,\N,25,106,\N,1.57,13.02,0.0,0.5,...,\N,\N,\N,0.00,420.0,22,31,0,0,0
41737,2,\N,\N,74,262,\N,2.31,15.39,0.0,0.5,...,\N,\N,\N,0.00,600.0,22,31,0,0,0
41738,2,\N,\N,65,79,\N,3.57,18.53,0.0,0.5,...,\N,\N,\N,0.75,960.0,22,31,0,0,0
41739,2,\N,\N,244,74,\N,4.64,23.19,0.0,0.5,...,\N,\N,\N,0.00,780.0,23,31,0,0,0
